# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

Для достижения цели нам необходимо:
1. изучить файл с данными;
2. разделить исходные данные на обучающую, валидационную и тестовую выборки;
3. исследовать качество разных моделей, меняя гиперпараметры;
4. проверить качество модели на тестовой выборке;
5. дополнительное задание: проверить модели на вменяемость;
6. Написать выводы.

## Откройте и изучите файл

Импортируем необходимые для работы библиотеки

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Прочитаем файл и сохраним в переменной df

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

Выведем на экран таблицу

In [3]:
display(df)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


Наименования столбцов корректные, но формат данных исправлять не будем.

Посмотрим общую информацию о данных

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Мы видим, что в столбцах одинакое количество данных. Из документации знаем, что
**сalls** — количество звонков,
**minutes** — суммарная длительность звонков в минутах,
**messages** — количество sms-сообщений,
**mb_used** — израсходованный интернет-трафик в Мб,
**is_ultra** — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

**Вывод**

С данными всё в порядке, можно переходить к обучению моделей.

## Разбейте данные на выборки

Теперь нам необходимо извлечь признаки

In [5]:
features = df.drop(['is_ultra'], axis=1)

И целевой признак

In [6]:
target = df['is_ultra']

Далее нам необходимо разделить датасет на обучающую, тестовую и валидационную выборки. На обучающую выборку оставим 60% данных, для тестовой и валидационной по 20% данных.

In [7]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.4, random_state=12345)

Теперь пополам разделим валидационную выборку, чтобы получить валидационную и тестовую

In [8]:
features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, target_valid, test_size=0.5, random_state=12345)

**Вывод**

Мы разбили данные на выборки, теперь можно переходить к исследованию модели.

## Исследуйте модели

В ходе выполнения этого шага, мы исследуем три модели:
- дерево решений
- случайный лес
- логистическую регрессию

### Дерево решений

Зададим параметры для дерева решений, напишем функцию, с помощью которой обучим модель и проверим её на валидационной выборке. Так как слишком высокое дерево решений имеет склонность к переобучению, а слишком низкое - к недообучению, установим параметры высоты от 1 до 5, как наиболее часто встречающиеся. Если при проверке дерева решений с этими гиперпараметрами увидим необходимость проверить более высокое дерево, то сделаем это следующим шагом.

In [10]:
for depth in range(1, 6):
    dtc_model = DecisionTreeClassifier(max_depth=depth, random_state=12345)
    dtc_model.fit(features_train, target_train)
    
    dtc_valid_predictions = dtc_model.predict(features_valid)
    accuracy = accuracy_score(target_valid, dtc_valid_predictions)
    print(f"DecisionTreeClassifier. max_depth = {depth}; accuracy = {accuracy}")

DecisionTreeClassifier. max_depth = 1; accuracy = 0.7542768273716952
DecisionTreeClassifier. max_depth = 2; accuracy = 0.7822706065318819
DecisionTreeClassifier. max_depth = 3; accuracy = 0.7853810264385692
DecisionTreeClassifier. max_depth = 4; accuracy = 0.7791601866251944
DecisionTreeClassifier. max_depth = 5; accuracy = 0.7791601866251944


Мы видим, что самый хороший результат показывает дерево с параметром depth = 3, до него высокое значение accuracy только у depth = 2, а после 3 значение accuracy начинает снижаться, поэтому проверять более высокие деревья не имеет смысла - они будут переобучаться.

### Случайный лес

Теперь обучим модель с помощью случайного леса. Посмотрим, как влияет на значение accuracy количество деревьев от 10 до 100.

In [12]:
for i in range(10, 101, 10):
    rfc_model = RandomForestClassifier(random_state=12345, n_estimators=i)
    rfc_model.fit(features_train, target_train)
    rfc_valid_predictions = rfc_model.predict(features_valid)
    score = accuracy_score( target_valid, rfc_valid_predictions)
    print(f'RandomForestClassifier, estimators={i}, score={score}')

RandomForestClassifier, estimators=10, score=0.7853810264385692
RandomForestClassifier, estimators=20, score=0.7869362363919129
RandomForestClassifier, estimators=30, score=0.7838258164852255
RandomForestClassifier, estimators=40, score=0.7838258164852255
RandomForestClassifier, estimators=50, score=0.7916018662519441
RandomForestClassifier, estimators=60, score=0.7853810264385692
RandomForestClassifier, estimators=70, score=0.7791601866251944
RandomForestClassifier, estimators=80, score=0.7822706065318819
RandomForestClassifier, estimators=90, score=0.7838258164852255
RandomForestClassifier, estimators=100, score=0.7853810264385692


Мы видим, что наилучший результат показывает лес, с количеством деревьев равным 50.

### Логистическая регрессия

Теперь рассмотрим модель логистической регрессии

In [13]:
log_model = LogisticRegression(random_state=12345)
log_model.fit(features_train, target_train)
log_predictions = log_model.predict(features_valid)
score = accuracy_score( target_valid, log_predictions)
print(f'Logistic Regression, score = {score}')

Logistic Regression, score = 0.7107309486780715


Логистическая регрессия показывает результат, не удовлетворяющий заданию, её использовать не будем.

**Вывод**

В ходе выполнения исследования моделей мы выяснили, что наилучший результат показывают:
1. RandomForestClassifier, estimators=50, accuracy = 0.79
2. RandomForestClassifier, estimators=10, accuracy = 0.78
3. DecisionTreeClassifier. max_depth = 3; accuracy = 0.78

Самый высокий показатель точности был у модели случайный лес с гиперпараметром n_estimators=50. Эту модель мы будем проверять на тестовой выборке.

## Проверьте модель на тестовой выборке

Проверим модель случайного леса  с гиперпараметром n_estimators=50 на тестовой выборке.

In [14]:
model = RandomForestClassifier(random_state=12345, n_estimators=50)
model.fit(features_train, target_train)
predictions = model.predict(features_test)

print(f"RandomForestClassifier. n_estimators = 50; accuracy = {accuracy_score(target_test, predictions)}")

RandomForestClassifier. n_estimators = 50; accuracy = 0.7931570762052877


**Вывод**

Мы видим, что случайный лес показывает хороший результат и на тестовой выборке, доля ошибок невелика. Мы выбрали верную модель, можно проверить её в работе на реальных данных. Аccuracy удовлетворяет требованиям.

## Вывод

Итак, в ходе выполнения проекта мы проверили три модели обучения: дерево решений, случайный лес и логистическую регрессию, из них выбрали ту модель, которая показала наилучший результат. Модель случайного леса с гиперпараметром n_estimators=50 показала на тестовой выборке accuracy = 0.79. Также хороший результат показывал случайный лес с 10 деревьями и дерево решений высотой 3.

Мы выяснили, что дерево решений довольно быстро начинает переобучаться, accuracy начало снижаться после 3. А вот модель случайнного леса оказалась меньше склонна к переобучению, так как везде был довольно высокий результат. Логистическая регрессия показала самый низкий результат, поэтому её мы не проверяли на тестовой выборке.